In [1]:
# MAE 1001: Introduction to Mechanical and Aerospace Engineering 
# Dice in Bias Experiment
### Instructor: 
# **Prof. Kartik Bulusu [MAE]**

#### Teaching Assistant: 
# **Preethi Siva Kumar [MAE-MS student]**

#### Learning Assistants: 
# **Nathan Janssen [MAE-Senior], Olivia Falciani [MAE-Senior], Shota Kakiuichi [MAE-Senior]**

#### Technical Support
# **Rithik Gunuganti [CS-MS student]**


# If you have any questions regarding Python, please feel free to send a slack message or 
# email to a member of the teaching team or come to office hours! We are happy to help!!

import dash
from dash import dcc, html, Input, Output, State
import plotly.express as px
import pandas as pd
import os
from datetime import datetime

# Function to generate CSV filename based on die name and current date
def get_csv_filename(die_name):
    today = datetime.now().strftime('%m/%d/%y')  # Format like mm/dd/yy
    clean_die = die_name.replace('_', '')  # Example: 'p_4' becomes 'p4'
    return f"{clean_die}_{today.replace('/', '')}.csv"  # e.g., p4_092925.csv

app = dash.Dash(__name__)
server = app.server  # If we want to deploy this in the future, for now we're running locally

# Global counter for entries (rolls submitted)
entry_count = 0
MAX_ENTRIES = 50

app.layout = html.Div([
    html.H2("Dice Roll Experiment Entry"),
    html.Div([
        html.Label('Last Name'),
        dcc.Input(id='last_name', type='text', value=''),
        html.Br(),
        html.Label('First Name'),
        dcc.Input(id='first_name', type='text', value=''),
        html.Br(),
        html.Label('Die Name (e.g., o_2 for orange die #2)'),
        dcc.Input(id='die_name', type='text', value=''),
        html.Br(),
        
        # Buttons for the face values (1 to 6)
        html.Div([
	    html.Label('Die Face Value'),
            html.Button('1', id='btn_1', n_clicks=0),
            html.Button('2', id='btn_2', n_clicks=0),
            html.Button('3', id='btn_3', n_clicks=0),
            html.Button('4', id='btn_4', n_clicks=0),
            html.Button('5', id='btn_5', n_clicks=0),
            html.Button('6', id='btn_6', n_clicks=0),
        ], style={'display': 'flex', 'gap': '10px'}),
        
        html.Div(id='error_msg', style={'color': 'red'}),
        html.Div(id='status_msg', style={'color': 'green'}),
        html.Div(id='terminate_msg', style={'color': 'blue'}),
    ]),
    html.H3("Histogram of Rolls for This Die"),
    dcc.Graph(id='histogram_plot'),

    # Hidden store to track button clicks
    dcc.Store(id='store_clicks', data={'btn_1': 0, 'btn_2': 0, 'btn_3': 0, 'btn_4': 0, 'btn_5': 0, 'btn_6': 0})
])

@app.callback(
    [Output('histogram_plot', 'figure'),
     Output('error_msg', 'children'),
     Output('status_msg', 'children'),
     Output('terminate_msg', 'children'),
     Output('store_clicks', 'data')],  # Output to reset the store
    [Input('btn_1', 'n_clicks'),
     Input('btn_2', 'n_clicks'),
     Input('btn_3', 'n_clicks'),
     Input('btn_4', 'n_clicks'),
     Input('btn_5', 'n_clicks'),
     Input('btn_6', 'n_clicks')],
    [State('last_name', 'value'),
     State('first_name', 'value'),
     State('die_name', 'value'),
     State('store_clicks', 'data')]  # Get the current state of button clicks
)
def update_output(btn_1, btn_2, btn_3, btn_4, btn_5, btn_6, last_name, first_name, die_name, clicks_data):
    global entry_count
    error = ""
    status = ""
    terminate = ""
    
    # Check which button was clicked and set the face_value accordingly
    face_value = None
    if btn_1 > clicks_data['btn_1']:
        face_value = 1
    elif btn_2 > clicks_data['btn_2']:
        face_value = 2
    elif btn_3 > clicks_data['btn_3']:
        face_value = 3
    elif btn_4 > clicks_data['btn_4']:
        face_value = 4
    elif btn_5 > clicks_data['btn_5']:
        face_value = 5
    elif btn_6 > clicks_data['btn_6']:
        face_value = 6

    # Update the clicks in the store after processing
    clicks_data = {'btn_1': btn_1, 'btn_2': btn_2, 'btn_3': btn_3, 'btn_4': btn_4, 'btn_5': btn_5, 'btn_6': btn_6}

    if face_value is None:
        # No button clicked yet
        empty_df = pd.DataFrame({'Face Value': []})
        fig = px.histogram(empty_df, x='Face Value', nbins=6, title='No data yet!')
        fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False})
        return fig, error, status, terminate, clicks_data

    # Validating inputs
    if not (last_name and first_name and die_name and face_value):
        error = "Please fill out all fields."
        df = pd.DataFrame()  # Empty for plot
    else:
        try:
            if face_value < 1 or face_value > 6:
                error = "Face value must be between 1 and 6."
            elif entry_count >= MAX_ENTRIES:
                error = "Maximum 50 entries reached. Please stop the app."
                terminate = "App will not accept more entries. Close the browser tab or stop the script."
            else:
                # Generate CSV filename
                data_file = get_csv_filename(die_name)

                # Load existing data if exists
                if os.path.exists(data_file):
                    df = pd.read_csv(data_file)
                else:
                    df = pd.DataFrame(columns=['Last Name', 'First Name', 'Die Name', 'Face Value'])

                # Append new row
                new_row = {
                    'Last Name': last_name,
                    'First Name': first_name,
                    'Die Name': die_name,
                    'Face Value': face_value
                }
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                df.to_csv(data_file, index=False)

                # Increment counter
                entry_count += 1
                status = f"Roll submitted! Total entries: {entry_count}. (Min 10 rolls recommended)"

                if entry_count >= MAX_ENTRIES:
                    terminate = "Reached 50 entries. No more submissions allowed. Stop the app and send the CSV file."

        except ValueError:
            error = "Face value must be an integer."

    # Create histogram from current data
    if not df.empty:
        fig = px.histogram(df, x='Face Value', nbins=6,
                           title=f"Histogram of Face Values for Die: {die_name}",
                           labels={'Face Value': 'Face Value (1-6)'},
                           category_orders={'Face Value': [1, 2, 3, 4, 5, 6]})
        fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
        fig.update_layout(bargap=0.1)
    else:
        # Empty plot with an empty DataFrame to avoid Plotly error
        empty_df = pd.DataFrame({'Face Value': []})
        fig = px.histogram(empty_df, x='Face Value', nbins=6, title='No data yet!')
        fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False})

    return fig, error, status, terminate, clicks_data

if __name__ == '__main__':
    app.run(debug=False)
